In [ ]:
import openai
import os

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY") 

def analyze_finances(income, spending_habits, savings_goal):
    """
    Basic logic rules to assess feasibility and concerns
    """
    fixed_expenses = 0.5 * income  
    discretionary = income - fixed_expenses
    goal_amount = extract_goal_amount(savings_goal)

    analysis = []
    if goal_amount > discretionary:
        analysis.append(f"User’s savings goal (${goal_amount}) may be too aggressive — discretionary income is about ${discretionary:.2f}.")
        goal_status = "too high"
    else:
        analysis.append(f"User’s savings goal (${goal_amount}) is achievable given discretionary income of about ${discretionary:.2f}.")
        goal_status = "achievable"

    if any(phrase in spending_habits.lower() for phrase in ["eat out", "restaurant", "fast food", "coffee shop"]):
        analysis.append("High discretionary spending on dining out detected.")
    if any(phrase in spending_habits.lower() for phrase in ["shop", "clothes", "gadgets", "luxury"]):
        analysis.append("High discretionary spending on shopping detected.")
    if any(phrase in spending_habits.lower() for phrase in ["not track", "rarely track", "don’t track", "don’t know where it goes"]):
        analysis.append("User does not track expenses.")

    return {
        "fixed_expenses": fixed_expenses,
        "discretionary": discretionary,
        "goal_amount": goal_amount,
        "goal_status": goal_status,
        "analysis_summary": " ".join(analysis)
    }

def extract_goal_amount(goal_text):
    """
    Simple heuristic to extract savings goal amount
    """
    import re
    match = re.search(r"\$?(\d+)", goal_text.replace(',', ''))
    if match:
        return float(match.group(1))
    else:
        # Default fallback if not specified clearly
        return 0.0

def build_gpt_prompt(user_data, analysis_data):
    """
    Compose the full prompt for GPT
    """
    prompt = f"""
User’s monthly income: ${user_data['income']}
User’s reported spending habits: {user_data['spending_habits']}
User’s savings goal: {user_data['savings_goal']}

Basic analysis:
- Estimated fixed living expenses: about ${analysis_data['fixed_expenses']:.2f}
- Remaining discretionary income: about ${analysis_data['discretionary']:.2f}
- User’s savings goal (${analysis_data['goal_amount']}) is {analysis_data['goal_status']}.
- Other observations: {analysis_data['analysis_summary']}

Please give the user:
1. A brief assessment of whether their savings goal is realistic.
2. 3–5 simple and specific money-saving suggestions based on their spending habits.
3. Encouragement to stay on track.

Keep the tone friendly, non-judgmental, and motivating.
"""
    return prompt.strip()

def get_finance_tips_from_gpt(prompt):
    """
    Call GPT-3.5 API with the crafted prompt
    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful and practical personal finance coach."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=400
    )
    return response.choices[0].message.content.strip()

def personal_finance_tip_generator(income, spending_habits, savings_goal):
    """
    Main function: takes inputs, applies rules, and outputs GPT advice
    """
    user_data = {
        "income": income,
        "spending_habits": spending_habits,
        "savings_goal": savings_goal
    }

    # Step 1: Basic analysis
    analysis_data = analyze_finances(income, spending_habits, savings_goal)

    # Step 2: Build prompt
    prompt = build_gpt_prompt(user_data, analysis_data)

    # Step 3: Get GPT response
    tips = get_finance_tips_from_gpt(prompt)

    return tips

if __name__ == "__main__":
    # Example user input
    income = 3500
    spending_habits = "I tend to eat out often, shop for clothes frequently, and rarely track my expenses."
    savings_goal = "Save $500 per month for an emergency fund."

    advice = personal_finance_tip_generator(income, spending_habits, savings_goal)
    print("\n📊 Personal Finance Tips:\n")
    print(advice)
